# Publications markdown generator for academicpages

Takes a TSV of publications with metadata and converts them for use with [academicpages.github.io](academicpages.github.io). This is an interactive Jupyter notebook ([see more info here](http://jupyter-notebook-beginner-guide.readthedocs.io/en/latest/what_is_jupyter.html)). The core python code is also in `publications.py`. Run either from the `markdown_generator` folder after replacing `publications.tsv` with one containing your data.

TODO: Make this work with BibTex and other databases of citations, rather than Stuart's non-standard TSV format and citation style.


## Data format

The TSV needs to have the following columns: pub_date, title, venue, excerpt, citation, site_url, and paper_url, with a header at the top. 

- `excerpt` and `paper_url` can be blank, but the others must have values. 
- `pub_date` must be formatted as YYYY-MM-DD.
- `url_slug` will be the descriptive part of the .md file and the permalink URL for the page about the paper. The .md file will be `YYYY-MM-DD-[url_slug].md` and the permalink will be `https://[yourdomain]/publications/YYYY-MM-DD-[url_slug]`

This is how the raw file looks (it doesn't look pretty, use a spreadsheet or other program to edit and create).

In [1]:
!cat publications.tsv

'cat' is not recognized as an internal or external command,
operable program or batch file.


## Import pandas

We are using the very handy pandas library for dataframes.

In [2]:
import pandas as pd

## Import TSV

Pandas makes this easy with the read_csv function. We are using a TSV, so we specify the separator as a tab, or `\t`.

I found it important to put this data in a tab-separated values format, because there are a lot of commas in this kind of data and comma-separated values can get messed up. However, you can modify the import statement, as pandas also has read_excel(), read_json(), and others.

In [34]:
publications = pd.read_csv("publications.tsv", sep="\t", dtype={'pub_date': str})
publications


,pub_date,authors,title,venue,venue_details,abbr_venue,category,excerpt,paper_url,code_url,slide_url,comment
0,2024,"Fei Wang, Zhenya Huang, Qi Liu, Enhong Chen, Y...",Dynamic Cognitive Diagnosis: An Educational Pr...,IEEE Transactions on Learning Technologies,16(3): 306-323,IEEE TLT,Journal,NaN,{{site.baseurl}}/files/DynamicCD_TLT_2023.pdf,https://github.com/LegionKing/Dynamic-Cognitiv...,NaN,NaN
1,2023,"Fei Wang, Qi Liu, Enhong Chen, Zhenya Huang, Y...",NeuralCD: a general framework for cognitive di...,IEEE Transactions on Knowledge and Data Engine...,35(8):8312-8327,IEEE TKDE,Journal,NaN,{{site.baseurl}}/files/NeuralCD_TKDE_2023.pdf,https://github.com/LegionKing/Neural_Cognitive...,NaN,NaN
2,2023,"Suojuan Zhang, Song Huang, Xiaohan Yu, Enhong ...",A generalized multi-skill aggregation method f...,World Wide Web,26(2): 585-614,WWW,Journal,NaN,https://link.springer.com/article/10.1007/s112...,NaN,NaN,NaN
3,2022,"Jiayu Liu, Fei Wang, Haiping Ma, Zhenya Huang,...",A probabilistic framework for temporal cogniti...,Journal of Computer Science and Technology,38(6):1203-1222,JCST,Journal,NaN,https://jcst.ict.ac.cn/en/article/pdf/preview/...,NaN,NaN,NaN
4,2019,"Wei Tong, Fei Wang, Qi Liu, Enhong Chen",Data driven prediction for the difficulty of m...,Journal of Computer Research and Development,56(5):1007-1019,CRD,Journal,NaN,http://staff.ustc.edu.cn/~qiliuql/files/Public...,NaN,NaN,NaN
5,2024,"Fei Wang, Qi Liu, Enhong Chen, Chuanren Liu, Z...",Unified Uncertainty Estimation for Cognitive D...,Proceedings of the ACM on Web Conference,3545-3554,WWW,Conference,NaN,{{site.baseurl}}/files/UCD_WWW_2024.pdf,https://github.com/LegionKing/UCD,NaN,NaN
6,2020,"Fei Wang, Qi Liu, Enhong Chen, Zhenya Huang, Y...",Neural cognitive diagnosis for intelligent edu...,Proceedings of the AAAI conference on artifici...,6153-6161,AAAI,Conference,NaN,{{site.baseurl}}/files/NeuralCD_AAAI_2020.pdf,https://github.com/bigdata-ustc/Neural_Cogniti...,{{site.baseurl}}/files/NeuralCD_AAAI_2020.pptx,NaN
7,2023,"Zheng Zhang, Qi Liu, Hao Jiang, Fei Wang, Yan ...",FairLISA: Fair User Modeling with Limited Sens...,Thirty-seventh Conference on Neural Informatio...,NaN,NeurIPS,Conference,NaN,https://openreview.net/pdf?id=uFpjPJMkv6,https://github.com/zhengz99/FairLISA,NaN,NaN
8,2023,"Sanshi Lei Yu, Qi Liu, Fei Wang, Yang Yu, Enho...",Federated News Recommendation with Fine-graine...,Proceedings of the 32nd ACM International Conf...,3073-3082,CIKM,Conference,NaN,https://dl.acm.org/doi/abs/10.1145/3583780.361...,NaN,NaN,NaN
9,2023,"Zihang Chen, Mengxiao Zhu, Fei Wang, Shuanghon...",Long-Term and Short-Term Perception in Knowled...,CCF Conference on Big Data,1-15,CCF bigdata,Conference,NaN,https://link.springer.com/chapter/10.1007/978-...,NaN,NaN,NaN


## Escape special characters

YAML is very picky about how it takes a valid string, so we are replacing single and double quotes (and ampersands) with their HTML encoded equivilents. This makes them look not so readable in raw format, but they are parsed and rendered nicely.

In [23]:
html_escape_table = {
    "&": "&amp;",
    '"': "&quot;",
    "'": "&apos;"
    }

def html_escape(text):
    """Produce entities within text."""
    return "".join(html_escape_table.get(c,c) for c in text)

# Text content of a Single paper
The function for creating the text content of a single paper according to a row of publications

In [47]:
def construct_paper_info_text(item):
	authors = item.authors.replace('Fei Wang', '**Fei Wang**')
	if item.category == 'Journal':
		ret = f"* {authors}. {html_escape(item.title)}. {html_escape(item.venue)}"
		if not pd.isna(item.abbr_venue):
			ret += f" ({item.abbr_venue})"
		if not pd.isna(item.venue_details):
			ret += f", {item.venue_details}"
		ret += f", {item.pub_date}."
	elif item.category == 'Conference':
		ret = f"* {authors}. {html_escape(item.title)}. {html_escape(item.abbr_venue)}'{item.pub_date}"
		if not pd.isna(item.venue_details):
			ret += f": {item.venue_details}"
		ret += f"."
	else:
		print("Unrecognized category!")
		print(item)
		exit()
	if not pd.isna(item.paper_url):
		ret += f" [[paper]]({item.paper_url})"
	if not pd.isna(item.code_url):
		ret += f" [[code]]({item.code_url})"
	if not pd.isna(item.slide_url):
		ret += f" [[slide]]({item.slide_url})"
	if not pd.isna(item.comment):
		ret += f"(**{item.comment}**)"
	return ret + '\n'


## Creating the markdown files

This is where the heavy lifting is done. This loops through all the rows in the TSV dataframe, then starts to concatentate a big string (```md```) that contains the markdown for each type. This is re-written by Fei Wang and is different from the original code that does the YAML metadata first and then does the description for the individual page.

In [48]:
import os
conf_papers, jour_papers = {}, {}
for year in publications['pub_date'].unique():
    conf_papers[year] = ""
    jour_papers[year] = ""

md = r"<!-- copy and paste the following content directly into the _pages/publications.md -->" + "\n"

for row, item in publications.iterrows():
    if item.category == 'Journal':
        jour_papers[item.pub_date] += construct_paper_info_text(item)
    else:
        conf_papers[item.pub_date] += construct_paper_info_text(item)

md += "## Journal Papers\n"
# sort the papers by pub_date in descending order
sorted_papers = sorted(jour_papers.items(), key=lambda x: x[0], reverse=True)
md = md + "".join([t[1] for t in sorted_papers])

md += "\n## Conference Papers\n"
sorted_papers = sorted(conf_papers.items(), key=lambda x: x[0], reverse=True)
md = md + "".join([t[1] for t in sorted_papers])

with open('publication_list.md', 'w') as o_f:
    o_f.write(md)